## BP Weekly Reporting
### Data Wrangling

Wrangle and repackage weekly data extracts from GA, Facebook and Google AdWords into a tidy dataset prepped for analysis.

In [1]:
# import packages
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from time import strptime
import datetime as dt
import os
import re

In [2]:
## read in FB export 10/29-11/4
Facebook = pd.read_csv('Bluprint_Weekly_Reporting_Template_Facebook.csv')
print(Facebook.info())
Facebook.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 14 columns):
Ad Set ID               191 non-null float64
Ad Set Name             191 non-null object
Ad ID                   191 non-null float64
Ad Name                 191 non-null object
Day                     191 non-null object
Platform                191 non-null object
Amount Spent (USD)      191 non-null float64
Impressions             191 non-null int64
Link Clicks             99 non-null float64
3-Second Video Views    63 non-null float64
Video Watches at 95%    59 non-null float64
Website Conversions     88 non-null float64
Reporting Starts        191 non-null object
Reporting Ends          191 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 21.0+ KB
None


,Ad Set ID,Ad Set Name,Ad ID,Ad Name,Day,Platform,Amount Spent (USD),Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions,Reporting Starts,Reporting Ends
0,2.384300e+16,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,2.384300e+16,Sew_Photo_FrenchSeam_StartStitching,10/30/18,instagram,1.34,119,NaN,NaN,NaN,NaN,10/30/18,10/30/18
1,2.384300e+16,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,2.384300e+16,Sew_Photo_FrenchSeam_StartStitching,10/29/18,messenger,0.41,19,NaN,NaN,NaN,NaN,10/29/18,10/29/18
2,2.384300e+16,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,2.384300e+16,Stitch_Video_Knit_:15_StartStitching,10/30/18,facebook,23.85,1137,4.0,117.0,27.0,3.0,10/30/18,10/30/18
3,2.384300e+16,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,2.384300e+16,Sew_Video_Sew_:15_StartStitching,10/29/18,facebook,41.22,2094,2.0,268.0,42.0,8.0,10/29/18,10/29/18
4,2.384300e+16,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,2.384300e+16,All_Video_DCComics,10/29/18,facebook,27.78,1301,3.0,218.0,27.0,1.0,10/29/18,10/29/18


In [3]:
# Drop unneeded columns 
Facebook = Facebook.filter(['Ad Set Name', 'Ad Name', 'Day', 'Platform','Amount Spent (USD)','Impressions','Link Clicks','3-Second Video Views','Video Watches at 95%','Website Conversions'])
Facebook.head()

,Ad Set Name,Ad Name,Day,Platform,Amount Spent (USD),Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions
0,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/30/18,instagram,1.34,119,NaN,NaN,NaN,NaN
1,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/29/18,messenger,0.41,19,NaN,NaN,NaN,NaN
2,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Stitch_Video_Knit_:15_StartStitching,10/30/18,facebook,23.85,1137,4.0,117.0,27.0,3.0
3,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Video_Sew_:15_StartStitching,10/29/18,facebook,41.22,2094,2.0,268.0,42.0,8.0
4,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,All_Video_DCComics,10/29/18,facebook,27.78,1301,3.0,218.0,27.0,1.0


In [4]:
# Change column names 
Facebook.columns = ['Placement', 'Creative', 'Date', 'Platform', 'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions']
Facebook.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/30/18,instagram,1.34,119,NaN,NaN,NaN,NaN
1,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/29/18,messenger,0.41,19,NaN,NaN,NaN,NaN
2,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Stitch_Video_Knit_:15_StartStitching,10/30/18,facebook,23.85,1137,4.0,117.0,27.0,3.0
3,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Video_Sew_:15_StartStitching,10/29/18,facebook,41.22,2094,2.0,268.0,42.0,8.0
4,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,All_Video_DCComics,10/29/18,facebook,27.78,1301,3.0,218.0,27.0,1.0


In [5]:
Facebook.Cost.sum()

2228.88

In [36]:
# read in Google export
Google = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google.csv')
Google.head()

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14,15,17,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Bluprint Weekly Reporting Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,"October 29, 2018 - November 4, 2018",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/29/18,NaN,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/30/18,NaN,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/31/18,NaN,$0.00,0,0,0,--,0


In [37]:
# bring header up from row index 1 and drop imported header and row index 0
Google.drop(Google.index[:1], inplace=True)
new_header = Google.iloc[0]
Google = Google[1:]
Google.columns = new_header
Google = Google.reset_index(drop=True)
Google.head()

1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/29/18,NaN,$0.00,0,0,0,--,0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/30/18,NaN,$0.00,0,0,0,--,0
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/31/18,NaN,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11/1/18,NaN,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11/2/18,NaN,$0.00,0,0,0,--,0


In [38]:
# filter out unwanted columns
Google = Google.filter(['Ad group','Ad','Day','Cost','Impressions','Clicks','Views','Video played to 100%','Conversions'])
print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63679 entries, 0 to 63678
Data columns (total 9 columns):
Ad group                63679 non-null object
Ad                      63679 non-null object
Day                     63679 non-null object
Cost                    63679 non-null object
Impressions             63679 non-null object
Clicks                  63679 non-null object
Views                   63679 non-null object
Video played to 100%    63679 non-null object
Conversions             63679 non-null object
dtypes: object(9)
memory usage: 4.4+ MB
None


1,Ad group,Ad,Day,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,$0.00,0,0,0,--,0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,$0.00,0,0,0,--,0
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,$0.00,0,0,0,--,0


In [39]:
# rename columns
Google['Platform'] = 'Google'
Google.columns = ['Placement','Creative','Date','Cost','Impressions','Clicks','Views','Completed Views','Conversions','Platform']
Google.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,$0.00,0,0,0,--,0,Google
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,$0.00,0,0,0,--,0,Google
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,$0.00,0,0,0,--,0,Google
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,$0.00,0,0,0,--,0,Google
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,$0.00,0,0,0,--,0,Google


In [40]:
# create function to strip non-numeric characters from a column
def strp_nonnum(col):
    '''strip string column of non numeric characters, keeping only digits and decimals'''
    col = col.str.extract('([\d]+.?\d*)')
    return col

# strip certain columns of non-numeric characters
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions']
for col in cols:
    Google[col] = strp_nonnum(Google[col])
    Google[col] = Google[col].str.replace(',','').astype(float)

print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63679 entries, 0 to 63678
Data columns (total 10 columns):
Placement          63679 non-null object
Creative           63679 non-null object
Date               63679 non-null object
Cost               63679 non-null float64
Impressions        63679 non-null float64
Clicks             32766 non-null float64
Views              32766 non-null float64
Completed Views    0 non-null float64
Conversions        32766 non-null float64
Platform           63679 non-null object
dtypes: float64(6), object(4)
memory usage: 4.9+ MB
None


,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,0.0,0.0,0.0,0.0,NaN,0.0,Google


In [41]:
Google.Cost.sum()

19253.93

In [42]:
Google = Google[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Google.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,Google,0.0,0.0,0.0,0.0,NaN,0.0


In [43]:
Google_2 = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google_Regs.csv')
Google_2.head()

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Day,Ad status,Auto-applied ad suggestion,Ad name,Device preference,Final URL,Mobile final URL,Display URL,Final URL suffix,Headline,...,Currency code,Cost,Impr.,Clicks,Views,Video played to 25%,Video played to 50%,Video played to 75%,Video played to 100%,Conversions
0,10/29/18,Enabled,No,--,All,http://mybluprint.com?cr_linkid=AW_DIS_GDN_MUL...,NaN,--,--,--,...,USD,$0.00,0,0,0,--,--,--,--,0
1,11/4/18,Enabled,No,--,All,http://mybluprint.com?cr_linkid=AW_DIS_GDN_MUL...,NaN,--,--,--,...,USD,$0.00,0,0,0,--,--,--,--,0
2,11/1/18,Enabled,No,--,All,http://mybluprint.com?cr_linkid=AW_DIS_GDN_MUL...,NaN,--,--,--,...,USD,$0.00,0,0,0,--,--,--,--,0
3,10/29/18,Enabled,No,--,All,http://mybluprint.com/start?cr_linkid=Cons_SEM...,NaN,--,--,--,...,USD,$19.76,286,73,0,--,--,--,--,3
4,10/30/18,Enabled,No,--,All,http://mybluprint.com/p/start-stitching?cr_lin...,NaN,--,--,--,...,USD,$0.00,0,0,0,--,--,--,--,0


In [44]:
# filter out unwanted columns
Google_2 = Google_2.filter(['Ad group','Ad name','Day','Cost','Impr.','Clicks','Views','Video played to 100%','Conversions'])
print(Google_2.info())
Google_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012480 entries, 0 to 1012479
Data columns (total 9 columns):
Ad group                1012480 non-null object
Ad name                 1012480 non-null object
Day                     1012480 non-null object
Cost                    1012480 non-null object
Impr.                   1012480 non-null object
Clicks                  1012480 non-null int64
Views                   1012480 non-null object
Video played to 100%    1012480 non-null object
Conversions             1012480 non-null int64
dtypes: int64(2), object(7)
memory usage: 69.5+ MB
None


,Ad group,Ad name,Day,Cost,Impr.,Clicks,Views,Video played to 100%,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/29/18,$0.00,0,0,0,--,0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,11/4/18,$0.00,0,0,0,--,0
2,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,11/1/18,$0.00,0,0,0,--,0
3,Cons_SEM_G_MULTISHOWCASE_Other_Other_Bluprint_...,--,10/29/18,$19.76,286,73,0,--,3
4,Cons_SEM_G_MULTISHOWCASE_Stitch_Embroidery_Sar...,--,10/30/18,$0.00,0,0,0,--,0


In [45]:
# rename columns
Google_2['Platform'] = 'Google'
Google_2.columns = ['Placement','Creative','Date','Cost','Impressions','Clicks','Views','Completed Views','Conversions','Platform']
print(Google_2.info())
Google_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012480 entries, 0 to 1012479
Data columns (total 10 columns):
Placement          1012480 non-null object
Creative           1012480 non-null object
Date               1012480 non-null object
Cost               1012480 non-null object
Impressions        1012480 non-null object
Clicks             1012480 non-null int64
Views              1012480 non-null object
Completed Views    1012480 non-null object
Conversions        1012480 non-null int64
Platform           1012480 non-null object
dtypes: int64(2), object(8)
memory usage: 77.2+ MB
None


,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/29/18,$0.00,0,0,0,--,0,Google
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,11/4/18,$0.00,0,0,0,--,0,Google
2,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,11/1/18,$0.00,0,0,0,--,0,Google
3,Cons_SEM_G_MULTISHOWCASE_Other_Other_Bluprint_...,--,10/29/18,$19.76,286,73,0,--,3,Google
4,Cons_SEM_G_MULTISHOWCASE_Stitch_Embroidery_Sar...,--,10/30/18,$0.00,0,0,0,--,0,Google


In [46]:
# strip certain columns of non-numeric characters
cols = ['Cost','Impressions','Views','Completed Views']
for col in cols:
    Google_2[col] = strp_nonnum(Google_2[col])
    Google_2[col] = Google_2[col].str.replace(',','').astype(float)

print(Google_2.info())
Google_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012480 entries, 0 to 1012479
Data columns (total 10 columns):
Placement          1012480 non-null object
Creative           1012480 non-null object
Date               1012480 non-null object
Cost               1012480 non-null float64
Impressions        106496 non-null float64
Clicks             1012480 non-null int64
Views              8192 non-null float64
Completed Views    12 non-null float64
Conversions        1012480 non-null int64
Platform           1012480 non-null object
dtypes: float64(4), int64(2), object(4)
memory usage: 77.2+ MB
None


,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/29/18,0.00,0.0,0,NaN,NaN,0,Google
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,11/4/18,0.00,0.0,0,NaN,NaN,0,Google
2,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,11/1/18,0.00,0.0,0,NaN,NaN,0,Google
3,Cons_SEM_G_MULTISHOWCASE_Other_Other_Bluprint_...,--,10/29/18,19.76,286.0,73,NaN,NaN,3,Google
4,Cons_SEM_G_MULTISHOWCASE_Stitch_Embroidery_Sar...,--,10/30/18,0.00,0.0,0,NaN,NaN,0,Google


In [47]:
Google_2['Clicks'] = Google_2['Clicks'].astype(float)
Google_2['Conversions'] = Google_2['Conversions'].astype(float)
Google_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012480 entries, 0 to 1012479
Data columns (total 10 columns):
Placement          1012480 non-null object
Creative           1012480 non-null object
Date               1012480 non-null object
Cost               1012480 non-null float64
Impressions        106496 non-null float64
Clicks             1012480 non-null float64
Views              8192 non-null float64
Completed Views    12 non-null float64
Conversions        1012480 non-null float64
Platform           1012480 non-null object
dtypes: float64(6), object(4)
memory usage: 77.2+ MB


In [48]:
Google_2 = Google_2[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Google_2.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/29/18,Google,0.00,0.0,0.0,NaN,NaN,0.0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,11/4/18,Google,0.00,0.0,0.0,NaN,NaN,0.0
2,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,11/1/18,Google,0.00,0.0,0.0,NaN,NaN,0.0
3,Cons_SEM_G_MULTISHOWCASE_Other_Other_Bluprint_...,--,10/29/18,Google,19.76,286.0,73.0,NaN,NaN,3.0
4,Cons_SEM_G_MULTISHOWCASE_Stitch_Embroidery_Sar...,--,10/30/18,Google,0.00,0.0,0.0,NaN,NaN,0.0


In [49]:
Google_2.Cost.sum()

11120.63000000001

In [50]:
Google = pd.concat([Google, Google_2], ignore_index=True)
Google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076159 entries, 0 to 1076158
Data columns (total 10 columns):
Placement          1076159 non-null object
Creative           1076159 non-null object
Date               1076159 non-null object
Platform           1076159 non-null object
Cost               1076159 non-null float64
Impressions        170175 non-null float64
Clicks             1045246 non-null float64
Views              40958 non-null float64
Completed Views    12 non-null float64
Conversions        1045246 non-null float64
dtypes: float64(6), object(4)
memory usage: 82.1+ MB


In [14]:
# read in Pinterest export
Pinterest = pd.read_csv('Bluprint_Weekly_Reporting_Template_Pinterest.csv')
Pinterest.head()

,Pin promotion ad group name,Promoted Pin name,Pin image,Promoted Pin ID,Promoted Pin description,Destination URL,Ad's campaign creative type,Spend,Total impressions,Checkout - cost per action,...,Paid average video watch time,Earned MRC video views,Earned video watched at 25%,Earned video watched at 50%,Earned video watched at 75%,Earned video watched at 95%,Earned video watched at 100%,Earned average video watch time,Ad holdout experiment.1,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,3.232354,429,0.000000,...,0,0,0,0,0,0,0,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434900,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,62.619226,17269,0.000000,...,0,0,0,0,0,0,0,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,20.088206,2551,0.000000,...,0,0,0,0,0,0,0,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434900,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,1003.948344,326831,0.000000,...,0,0,0,0,0,0,0,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,80.617012,11609,80.617012,...,0,0,0,0,0,0,0,0,0,10/20/18


In [15]:
# filter out unwanted columns, more efficient than dropping due to quantity of columns
Pinterest = Pinterest.filter(['Pin promotion ad group name','Promoted Pin name','Spend','Total impressions','Total clicks','Total conversions','Total MRC video views','Total video watched at 100%','Date'])
Pinterest.head()

,Pin promotion ad group name,Promoted Pin name,Spend,Total impressions,Total clicks,Total conversions,Total MRC video views,Total video watched at 100%,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18


In [16]:
# rename columns
Pinterest.columns = ['Placement', 'Creative', 'Cost', 'Impressions', 'Clicks', 'Conversions', 'Views', 'Completed Views','Date']
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Views,Completed Views,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18


In [17]:
# add columns to match FB df
Pinterest['Platform'] = 'Pinterest'
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Views,Completed Views,Date,Platform
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18,Pinterest
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18,Pinterest
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18,Pinterest
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18,Pinterest
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18,Pinterest


In [18]:
# reorder dataframe columns to make consistent with Facebook dataframe
Pinterest = Pinterest[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Pinterest.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/15/18,Pinterest,3.232354,429,3,0,0,44
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,10/15/18,Pinterest,62.619226,17269,47,0,0,53
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/19/18,Pinterest,20.088206,2551,12,0,0,99
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,10/19/18,Pinterest,1003.948344,326831,656,0,0,155
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/20/18,Pinterest,80.617012,11609,42,0,0,417


In [51]:
# read in Google Analytics export
GoogleAnalytics = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google_Analytics_Raw.csv')
GoogleAnalytics.head()

,# ----------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,# A-01 Bluprint Web Prod - All,NaN,NaN,NaN,NaN,NaN,NaN
1,# LP Visitor Source_Weekly Reporting Pull,NaN,NaN,NaN,NaN,NaN,NaN
2,# 20181029-20181104,NaN,NaN,NaN,NaN,NaN,NaN
3,# ----------------------------------------,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# bring up row index 5 as header and drop row indices 0-4
GoogleAnalytics.drop(GoogleAnalytics.index[:5], inplace=True)
new_header = GoogleAnalytics.iloc[0]
GoogleAnalytics = GoogleAnalytics[1:]
GoogleAnalytics.columns = new_header
GoogleAnalytics = GoogleAnalytics.reset_index(drop=True)
GoogleAnalytics.head()

5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
0,114964,20181031,"13,327","12,029","7,932",613,143
1,112180,20181104,"7,189","2,727","2,861",73,84
2,112180,20181103,"6,595","2,378","2,390",78,86
3,112180,20181102,"4,134","1,171","1,155",44,49
4,114964,20181101,"4,116","3,323","2,496",182,44


In [53]:
GoogleAnalytics.columns = ['MAID','Date','Sessions','New Users','Bounces','Registrations','Subscriptions']
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID      Date Sessions New Users Bounces Registrations Subscriptions
0  114964  20181031   13,327    12,029   7,932           613           143
1  112180  20181104    7,189     2,727   2,861            73            84
2  112180  20181103    6,595     2,378   2,390            78            86
3  112180  20181102    4,134     1,171   1,155            44            49
4  114964  20181101    4,116     3,323   2,496           182            44
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 7 columns):
MAID             3096 non-null object
Date             3096 non-null object
Sessions         3096 non-null object
New Users        3096 non-null object
Bounces          3096 non-null object
Registrations    3096 non-null object
Subscriptions    3096 non-null object
dtypes: object(7)
memory usage: 169.4+ KB


In [54]:
# bring date column to datetime dtype
GoogleAnalytics['Date'] = pd.to_datetime(GoogleAnalytics['Date'], format = '%Y/%m/%d')

# strip non-numeric characters from certain columns
cols = ['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions']
for col in cols:
    GoogleAnalytics[col] = strp_nonnum(GoogleAnalytics[col])
    GoogleAnalytics[col] = GoogleAnalytics[col].str.replace(',','')
    GoogleAnalytics[col] = pd.to_numeric(GoogleAnalytics[col], errors='coerce')
    
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,114964,2018-10-31,13327,12029,7932,613,143
1,112180,2018-11-04,7189,2727,2861,73,84
2,112180,2018-11-03,6595,2378,2390,78,86
3,112180,2018-11-02,4134,1171,1155,44,49
4,114964,2018-11-01,4116,3323,2496,182,44


In [55]:
# aggregate by MAID and Date, summing remaining columns
GoogleAnalytics = GoogleAnalytics.groupby(['MAID','Date']).sum().reset_index()
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,103706,2018-10-29,5,5,0,0,0
1,103706,2018-10-30,5,4,4,0,0
2,103706,2018-10-31,2,2,2,0,0
3,103706,2018-11-01,1,1,1,0,0
4,103706,2018-11-03,3,2,3,0,0


In [65]:
##MERGE DATA SETS
all_data = pd.concat([Google, Facebook], ignore_index=True)
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,Google,0.0,0.0,0.0,0.0,NaN,0.0


In [66]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076350 entries, 0 to 1076349
Data columns (total 10 columns):
Placement          1076350 non-null object
Creative           1076350 non-null object
Date               1076350 non-null object
Platform           1076350 non-null object
Cost               1076350 non-null float64
Impressions        170366 non-null float64
Clicks             1045345 non-null float64
Views              41021 non-null float64
Completed Views    71 non-null float64
Conversions        1045334 non-null float64
dtypes: float64(6), object(4)
memory usage: 82.1+ MB


In [67]:
# create new columns and categorize
all_data['MAID'] = all_data['Placement'].str.extract('(_1\d\d\d\d\d)', expand=True)
all_data['MAID'] = all_data['MAID'].str.strip('_')
all_data['Funnel'] = all_data['Placement'].str.split('_').str[0]
#all_data['Tactic'] = all_data['Placement'].str.split('_').str[1]
all_data['Publisher'] = all_data['Placement'].str.split('_').str[2]
all_data['Hook'] = all_data['Placement'].str.split('_').str[3]
all_data['Mega'] = all_data['Placement'].str.split('_').str[4]
all_data['Micro'] = all_data['Placement'].str.split('_').str[5]
all_data['Campaign'] = all_data['Placement'].str.split('_').str[6]
all_data['Targ/aud/pin'] = all_data['Placement'].str.split('_').str[7]
all_data['Keyword'] = all_data['Placement'].str.split('_').str[8]
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/29/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/30/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/31/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/1/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,11/2/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res


In [68]:
all_data.groupby('Mega')['Cost'].nunique()

Mega
All                     128
BP CA                     1
Bluprint LAL              1
Bluprint RM               1
Bluprint RM-Masthead      1
Bluprint SV               1
CI                        1
Compose                 298
Craftsy LAL               1
Craftsy SV                1
Dwell                    29
Engagement                1
KUWK                      1
Lookalike Audience        1
Make                    183
Move                     96
Other                   155
RM                        1
Savor                   157
Shahs of Sunset           5
Site Visitors             1
Stitch                  619
Together                 25
VV                        1
Name: Cost, dtype: int64

In [69]:
all_data['Funnel'].unique()

array(['CONV', 'Conv', 'AW', ..., 'Blueprint Paper Crafting Tutorials',
       'Blueprint Pastry Courses',
       'Blueprint Knitting And Crochet Training'], dtype=object)

In [70]:
all_data['Publisher'].unique()

array(['GDN', 'G', 'YT', nan, 'GreenLivingEnthusiasts',
       'Do-It-Yourselfers', 'BargainHunters',
       'BluprintRegisteredUsersNew', 'CakeRegistered', 'SewingRegistered',
       'KnittingRegistered', 'PageViews>1', 'Pageviews>1',
       'QuiltingRegistered', 'AllVisitors', 'SiteVisitorsStart',
       'CraftsyYTSubscribers', 'FB'], dtype=object)

In [71]:
all_data = all_data.loc[(all_data['Publisher'] == 'GDN') | (all_data['Publisher'] == 'G') | (all_data['Publisher'] == 'YT') | (all_data['Publisher'] == 'FB')]

In [72]:
all_data['Publisher'].unique()

array(['GDN', 'G', 'YT', 'FB'], dtype=object)

In [73]:
# rename categories for Funnel and Publisher columns
all_data['Funnel'] = all_data['Funnel'].replace({'AW' : 'Awareness', 'Cons' : 'Consideration', 'CONS' : 'Consideration', 'Conv' : 'Conversion', 'CONV' : 'Conversion'})
all_data['Publisher'] = all_data['Publisher'].replace({'G' : 'Google Search', 'GDN' : 'Google Display Network', 'FB' : 'Facebook', 'YT' : 'YouTube'})
all_data['Mega'] = all_data['Mega'].replace({'All':'All/Other','Other':'All/Other','Lookalike Audience':'All/Other','VV':'All/Other','Craftsy LAL':'All/Other','Craftsy SV':'All/Other','Bluprint LAL':'All/Other','Bluprint RM':'All/Other','Bluprint RM-Masthead':'All/Other','Bluprint SV':'All/Other','CI':'All/Other','KUWK':'All/Other','BP CA':'All/Other','Engagement':'All/Other','RM':'All/Other','Shahs of Sunset':'All/Other','Site Visitors':'All/Other'})
print(all_data['Funnel'].unique())
print(all_data['Publisher'].unique())
all_data['Mega'].unique()

['Conversion' 'Awareness' 'Consideration']
['Google Display Network' 'Google Search' 'YouTube' 'Facebook']


array(['Make', 'Savor', 'Move', 'All/Other', 'Stitch', 'Dwell', 'Compose',
       'Together'], dtype=object)

In [74]:
all_data.loc[(all_data['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword


In [50]:
all_data = all_data.loc[(all_data['Funnel'] == 'Awareness') | (all_data['Funnel'] == 'Consideration') | (all_data['Funnel'] == 'Conversion')]
all_data.loc[(all_data['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword


In [75]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126653 entries, 0 to 1076349
Data columns (total 19 columns):
Placement          126653 non-null object
Creative           126653 non-null object
Date               126653 non-null object
Platform           126653 non-null object
Cost               126653 non-null float64
Impressions        126653 non-null float64
Clicks             95648 non-null float64
Views              34318 non-null float64
Completed Views    63 non-null float64
Conversions        95637 non-null float64
MAID               125806 non-null object
Funnel             126653 non-null object
Publisher          126653 non-null object
Hook               126653 non-null object
Mega               126653 non-null object
Micro              126653 non-null object
Campaign           126632 non-null object
Targ/aud/pin       126590 non-null object
Keyword            126590 non-null object
dtypes: float64(6), object(13)
memory usage: 19.3+ MB


In [76]:
Facebook['Platform'].unique()

array(['instagram', 'messenger', 'facebook', 'audience_network'],
      dtype=object)

In [77]:
all_data.loc[all_data['Platform'] == 'instagram', 'Publisher'] = 'Instagram'
all_data['Publisher'].unique()

array(['Google Display Network', 'Google Search', 'YouTube', 'Instagram',
       'Facebook'], dtype=object)

In [78]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126653 entries, 0 to 1076349
Data columns (total 19 columns):
Placement          126653 non-null object
Creative           126653 non-null object
Date               126653 non-null object
Platform           126653 non-null object
Cost               126653 non-null float64
Impressions        126653 non-null float64
Clicks             95648 non-null float64
Views              34318 non-null float64
Completed Views    63 non-null float64
Conversions        95637 non-null float64
MAID               125806 non-null object
Funnel             126653 non-null object
Publisher          126653 non-null object
Hook               126653 non-null object
Mega               126653 non-null object
Micro              126653 non-null object
Campaign           126632 non-null object
Targ/aud/pin       126590 non-null object
Keyword            126590 non-null object
dtypes: float64(6), object(13)
memory usage: 19.3+ MB


In [79]:
all_data.loc[all_data['Placement'].str.split('_').str[1] == 'YT', 'Publisher'] = 'YouTube'

In [80]:
# calculate daily impressions by MAID
imp_by_MAID = all_data.groupby(by=['MAID', 'Date'])['Impressions'].sum().reset_index()
imp_by_MAID.head()

,MAID,Date,Impressions
0,111868,10/29/18,116.0
1,111868,10/30/18,125.0
2,111868,10/31/18,163.0
3,111868,11/1/18,148.0
4,111868,11/2/18,169.0


In [81]:
imp_by_MAID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 3 columns):
MAID           1065 non-null object
Date           1065 non-null object
Impressions    1065 non-null float64
dtypes: float64(1), object(2)
memory usage: 25.0+ KB


In [82]:
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,103706,2018-10-29,5,5,0,0,0
1,103706,2018-10-30,5,4,4,0,0
2,103706,2018-10-31,2,2,2,0,0
3,103706,2018-11-01,1,1,1,0,0
4,103706,2018-11-03,3,2,3,0,0


In [83]:
GoogleAnalytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 7 columns):
MAID             3096 non-null object
Date             3096 non-null datetime64[ns]
Sessions         3096 non-null int64
New Users        3096 non-null int64
Bounces          3096 non-null int64
Registrations    3096 non-null int64
Subscriptions    3096 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 169.4+ KB


In [84]:
# merge imp_by_MAID and GoogleAnalytics
imp_by_MAID['Date'] = pd.to_datetime(imp_by_MAID['Date'])
GoogleAnalytics = pd.merge(GoogleAnalytics, imp_by_MAID, how='inner', on=['MAID', 'Date'])
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID       Date  Sessions  New Users  Bounces  Registrations  \
0  111868 2018-10-29        46         32       21              2   
1  111868 2018-10-30        50         37       29              1   
2  111868 2018-10-31        51         32       27              0   
3  111868 2018-11-01        53         35       29              1   
4  111868 2018-11-02        45         33       28              0   

   Subscriptions  Impressions  
0              1        116.0  
1              0        125.0  
2              0        163.0  
3              0        148.0  
4              2        169.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 691 entries, 0 to 690
Data columns (total 8 columns):
MAID             691 non-null object
Date             691 non-null datetime64[ns]
Sessions         691 non-null int64
New Users        691 non-null int64
Bounces          691 non-null int64
Registrations    691 non-null int64
Subscriptions    691 non-null int64
Impressions      691 non-nu

In [85]:
##CALCULATE COLUMNS TO BE JOINED INTO OVERALL DATASET
GoogleAnalytics['Session Rate'] = GoogleAnalytics['Sessions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['New Session Rate'] = GoogleAnalytics['New Users'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Bounce Rate'] = GoogleAnalytics['Bounces'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Reg Rate'] = GoogleAnalytics['Registrations'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Sub Rate'] = GoogleAnalytics['Subscriptions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalyticsRates = GoogleAnalytics
GoogleAnalyticsRates.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions,Impressions,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-29,46,32,21,2,1,116.0,0.396552,0.695652,0.456522,0.017241,0.008621
1,111868,2018-10-30,50,37,29,1,0,125.0,0.400000,0.740000,0.580000,0.008000,0.000000
2,111868,2018-10-31,51,32,27,0,0,163.0,0.312883,0.627451,0.529412,0.000000,0.000000
3,111868,2018-11-01,53,35,29,1,0,148.0,0.358108,0.660377,0.547170,0.006757,0.000000
4,111868,2018-11-02,45,33,28,0,2,169.0,0.266272,0.733333,0.622222,0.000000,0.011834


In [86]:
GoogleAnalyticsRates.drop(columns=['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions', 'Impressions'], axis=1, inplace=True)
GoogleAnalyticsRates.head()

,MAID,Date,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-29,0.396552,0.695652,0.456522,0.017241,0.008621
1,111868,2018-10-30,0.400000,0.740000,0.580000,0.008000,0.000000
2,111868,2018-10-31,0.312883,0.627451,0.529412,0.000000,0.000000
3,111868,2018-11-01,0.358108,0.660377,0.547170,0.006757,0.000000
4,111868,2018-11-02,0.266272,0.733333,0.622222,0.000000,0.011834


In [87]:
##JOIN NEW GOOGLE ANALYTICS TO ALL_DATA_MERGE
all_data['Date'] = pd.to_datetime(all_data['Date'])
final_data_merge = pd.merge(all_data, GoogleAnalyticsRates, how='inner', on=['MAID', 'Date'])
print(final_data_merge.info())
final_data_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102460 entries, 0 to 102459
Data columns (total 24 columns):
Placement           102460 non-null object
Creative            102460 non-null object
Date                102460 non-null datetime64[ns]
Platform            102460 non-null object
Cost                102460 non-null float64
Impressions         102460 non-null float64
Clicks              77396 non-null float64
Views               26757 non-null float64
Completed Views     63 non-null float64
Conversions         77385 non-null float64
MAID                102460 non-null object
Funnel              102460 non-null object
Publisher           102460 non-null object
Hook                102460 non-null object
Mega                102460 non-null object
Micro               102460 non-null object
Campaign            102456 non-null object
Targ/aud/pin        102456 non-null object
Keyword             102456 non-null object
Session Rate        102460 non-null float64
New Session Rate    1

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.153846,0.846154,NaN,NaN
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.153846,0.846154,NaN,NaN
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.153846,0.846154,NaN,NaN
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.153846,0.846154,NaN,NaN
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_BPRM_Res_20180...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,BPRM,Res,0.0,0.153846,0.846154,NaN,NaN


In [88]:
# create new columns by multiplying others
final_data_merge['Sessions'] = final_data_merge['Impressions'].mul(final_data_merge['Session Rate'])
final_data_merge['New Sessions'] = final_data_merge['Sessions'].mul(final_data_merge['New Session Rate'])
final_data_merge['Bounces'] = final_data_merge['Sessions'].mul(final_data_merge['Bounce Rate'])
final_data_merge['Registrations'] = final_data_merge['Impressions'].mul(final_data_merge['Reg Rate'])
final_data_merge['Subscriptions'] = final_data_merge['Impressions'].mul(final_data_merge['Sub Rate'])
final_data_merge.drop(columns=['Session Rate', 'New Session Rate', 'Bounce Rate', 'Reg Rate', 'Sub Rate'], axis=1, inplace=True)
final_data_merge.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_BPRM_Res_20180...,--,2018-10-29,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,BPRM,Res,0.0,0.0,0.0,NaN,NaN


In [89]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102460 entries, 0 to 102459
Data columns (total 24 columns):
Placement          102460 non-null object
Creative           102460 non-null object
Date               102460 non-null datetime64[ns]
Platform           102460 non-null object
Cost               102460 non-null float64
Impressions        102460 non-null float64
Clicks             77396 non-null float64
Views              26757 non-null float64
Completed Views    63 non-null float64
Conversions        77385 non-null float64
MAID               102460 non-null object
Funnel             102460 non-null object
Publisher          102460 non-null object
Hook               102460 non-null object
Mega               102460 non-null object
Micro              102460 non-null object
Campaign           102456 non-null object
Targ/aud/pin       102456 non-null object
Keyword            102456 non-null object
Sessions           102460 non-null float64
New Sessions       102460 non-null float6

In [90]:
##ADD REMAINING COLUMNS
final_data_merge['Start Date'] = final_data_merge.groupby(['Placement'])['Date'].transform(min)
final_data_merge['Video v Static'] = np.where(final_data_merge['Views']>=1, 'Video', 'Static')
final_data_merge['Impressions with Video View'] = np.where(final_data_merge['Views']>=1, final_data_merge['Impressions'], 0)
final_data_merge['Reporting Month'] = final_data_merge['Date'].dt.strftime('%B')
final_data_merge['Week Start'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)
final_data_merge['Reporting Period'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)

In [91]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102460 entries, 0 to 102459
Data columns (total 30 columns):
Placement                      102460 non-null object
Creative                       102460 non-null object
Date                           102460 non-null datetime64[ns]
Platform                       102460 non-null object
Cost                           102460 non-null float64
Impressions                    102460 non-null float64
Clicks                         77396 non-null float64
Views                          26757 non-null float64
Completed Views                63 non-null float64
Conversions                    77385 non-null float64
MAID                           102460 non-null object
Funnel                         102460 non-null object
Publisher                      102460 non-null object
Hook                           102460 non-null object
Mega                           102460 non-null object
Micro                          102460 non-null object
Campaign          

In [92]:
#REORGANIZE REMAINING COLUMNS
final_data_merge = final_data_merge[['Funnel', 'Publisher', 'Hook', 'Mega', 'Micro', 
                                     'Campaign', 'Targ/aud/pin', 'Keyword', 'Start Date', 'MAID', 
                                     'Video v Static', 'Impressions with Video View', 'Week Start', 
                                     'Reporting Month', 'Reporting Period', 'Placement', 'Creative', 'Date', 
                                     'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions', 
                                     'Sessions', 'New Sessions', 'Bounces', 'Registrations', 'Subscriptions']]
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102460 entries, 0 to 102459
Data columns (total 29 columns):
Funnel                         102460 non-null object
Publisher                      102460 non-null object
Hook                           102460 non-null object
Mega                           102460 non-null object
Micro                          102460 non-null object
Campaign                       102456 non-null object
Targ/aud/pin                   102456 non-null object
Keyword                        102456 non-null object
Start Date                     102460 non-null datetime64[ns]
MAID                           102460 non-null object
Video v Static                 102460 non-null object
Impressions with Video View    102460 non-null float64
Week Start                     102460 non-null datetime64[ns]
Reporting Month                102460 non-null object
Reporting Period               102460 non-null datetime64[ns]
Placement                      102460 non-null object


In [93]:
final_data_merge['Date'].unique()

array(['2018-10-29T00:00:00.000000000', '2018-10-30T00:00:00.000000000',
       '2018-10-31T00:00:00.000000000', '2018-11-01T00:00:00.000000000',
       '2018-11-02T00:00:00.000000000', '2018-11-04T00:00:00.000000000',
       '2018-11-03T00:00:00.000000000'], dtype='datetime64[ns]')

In [94]:
final_data_merge['Targ/aud/pin'].unique()

array(['RM', 'BPRM', 'EG', 'JessicaLong', 'Celebrity', 'Celebrrity',
       'Generic', 'FabioViviani', 'E', 'Accessories', 'Doodle-Wars',
       'BrookeLark', 'JoshuaVergara', 'JoshuaJohnRussell', 'BMM', 'CA',
       'AA', 'IM', 'SV', 'LAL', 'SV LAL', 'vv', 'VV', 'aa', 'im', 'CI',
       nan, 'WCA', 'RT'], dtype=object)

In [95]:
final_data_merge.Cost.sum()

27917.490000000005

In [96]:
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions','Sessions','New Sessions','Bounces','Registrations','Subscriptions']
for col in cols:
    final_data_merge[col].fillna(0)

In [97]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-29,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-29,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-29,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
3,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-29,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
4,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,BPRM,Res,2018-10-29,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [98]:
final_data_merge.to_csv('weekly_data_merge.csv')

### To Do


✅ drop 'Tactic' column 

✅ rename 'Video Completion' to 'Completed Views'

- audit columns for:
    - data type
    - quality
    - etc.
    
✅ Refine 'Mega' column to only include unique values, not duplicates with slight variations OR assign "All/Other" properly

In [74]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
3,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
4,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,BPRM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
